In [1]:
import pandas as pd
from datetime import datetime
import requests
import time 
from requests_html import HTMLSession
# from requests.packages.urllib3.exceptions import InsecureRequestWarning
from PIL import Image
from io import StringIO, BytesIO

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import display, clear_output


C:\Users\Kalim\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
! pip install --upgrade selenium

  Using cached urllib3-1.26.8-py2.py3-none-any.whl (138 kB)
  Using cached sniffio-1.2.0-py3-none-any.whl (10 kB)

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

requests 2.24.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.8 which is incompatible.



  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
  Attempting uninstall: selenium
    Found existing installation: selenium 3.9.0
    Uninstalling selenium-3.9.0:
      Successfully uninstalled selenium-3.9.0


In [2]:
s = Service('chromedriver')
driver = webdriver.Chrome(service=s)
pages = 100
single_page_urls = []
page_cnt, url_cnt = 0, 0
for p in range(1, pages+1):
    bikroy_url = f'https://bikroy.com/en/ads/bangladesh/mobiles?sort=date&order=desc&buy_now=0&urgent=0&page={p}'
    print(bikroy_url)
    driver.get(bikroy_url)
    time.sleep(3)
    page_cnt += 1
    for i in range (1, 28):
        try:
            single_url = driver.find_element(by=By.XPATH, value=f'//*[@id="app-wrapper"]/div[1]/div[3]/div/div[2]/div[4]/div[2]/div/div[1]/div[1]/ul/li[{i}]/a').get_attribute('href')
            single_page_urls.append(single_url)
            url_cnt += 1
            print("Page: " + str(page_cnt) + " URL: " + str(url_cnt))
        except:
            pass
    
    clear_output(wait=True)
# driver.maximize_window()

https://bikroy.com/en/ads/bangladesh/mobiles?sort=date&order=desc&buy_now=0&urgent=0&page=100
Page: 100 URL: 2242
Page: 100 URL: 2243
Page: 100 URL: 2244
Page: 100 URL: 2245
Page: 100 URL: 2246
Page: 100 URL: 2247
Page: 100 URL: 2248
Page: 100 URL: 2249
Page: 100 URL: 2250
Page: 100 URL: 2251
Page: 100 URL: 2252
Page: 100 URL: 2253
Page: 100 URL: 2254
Page: 100 URL: 2255
Page: 100 URL: 2256
Page: 100 URL: 2257
Page: 100 URL: 2258
Page: 100 URL: 2259
Page: 100 URL: 2260
Page: 100 URL: 2261
Page: 100 URL: 2262
Page: 100 URL: 2263
Page: 100 URL: 2264
Page: 100 URL: 2265
Page: 100 URL: 2266
Page: 100 URL: 2267
Page: 100 URL: 2268


In [5]:
len(single_page_urls)

2268

In [13]:
with open('mobile_product_urls.txt', 'w') as f:
    for s in single_page_urls:
        f.write(s)
        f.write('\n')

In [6]:
def title_scrap(driver):
    title = driver.find_element(by=By.TAG_NAME, value='h1').text
    #     //*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[1]/div[1]/div/div[1]/div/div[1]/div[2]/div/div/h1
    #     //*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[1]/div[1]/div/div[1]/div/div[1]/div[2]/div/div/h1

    return title

In [7]:
def image_scarp(driver):
    try:
        images = driver.find_element(by=By.XPATH, value='//*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[3]/div[2]/div/div[1]/div/div[1]/div/div/ul')
        img_path = []
        for img in images.find_elements(by=By.TAG_NAME,value='img'):
            img_urls = img.get_attribute('src')

            img_urls_firstname = '/'.join(img_urls.split('/')[:-3])
            img_urls_rename = img_urls_firstname + '/620/466/fitted.jpg'
    #         print(img_urls_rename)
            img_name = '@'.join(img_urls_firstname.split('/')[-2:])

            r = requests.get(img_urls_rename)
            i = Image.open(BytesIO(r.content))
            i.save(f"img/mobile/{img_name}.jpg")
            img_path.append(img_name)
        return img_path
    except:
        print('No images')        
        return 'N/A'

In [8]:
def meta_data_scrap(driver):
    try:
        meta_data = driver.find_element(by=By.XPATH, value='//*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[3]/div[2]/div/div[1]/div/div[2]/div[2]')

        md = meta_data.find_elements(by=By.TAG_NAME,value='div')
        meta_dic = {}
        for m in md:
            m = m.text
            if '\n' in m:
                key, val = m.split('\n')[0].replace(':', ''), m.split('\n')[1]
                meta_dic[key] = val
        print("Meta Data: ", meta_dic)
        return meta_dic
    except:
        print('No meta data found')        
        return 'N/A'

In [9]:
def show_more_click(driver):    
    try:
        s_driver.find_element(by=By.CLASS_NAME, value='show-text--13FcL').click()
#         s_driver.find_element(by=By.XPATH, value='//*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[3]/div[2]/div/div[1]/div/div[2]/div[3]/div/div[2]/div/div[2]/button').click()
        time.sleep(2)
    except:
        print("No 'Show more' button")

In [10]:
def feature_scrap(driver):
    try:
        feature = driver.find_element(by=By.CLASS_NAME, value='text--Xap4O').text
#         feature = s_driver.find_element(by=By.XPATH, value='//*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[1]/div[1]/div/div[1]/div/div[2]/div[4]/p').text
#         feature = feature.find_elements(by=By.TAG_NAME,value='p').text
        print('Features: ', feature)
        return feature
    except:
        print("No feature")        
        return 'N/A'

In [11]:
def description_scrap(driver):
    # desc = s_driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div[2]/div[2]/div[3]/div[2]/div/div[1]/div/div[2]/div[3]/div/div[2]/div/div[1]/div/ul/div')
    # description[0].text
    # description = [d.text.replace("'\uf076", "").replace("''\uf0d8", "") for d in desc.find_elements_by_tag_name('p')]
    # print(description)

    try:
        desc = s_driver.find_element(by=By.XPATH, value='//*[@id="collapsible-content-0"]/ul/div').text
        print("Description: ", desc)
        return desc
    except:
        print("No description")        
        return 'N/A'

In [ ]:
op = webdriver.ChromeOptions()
op.add_argument('headless')
s = Service('chromedriver')
s_driver = webdriver.Chrome(service=s, options=op)

cnt = 1
titles, img_lst, meta_lst,  feature_lst, desc_lst = [[]]*5 
for s_u in single_page_urls[:3]:
    print("Count: ", cnt)
    s_driver.get(s_u)
    time.sleep(3)

    # single page data scrapping
    s_title = title_scrap(s_driver)
    print("Title: ", s_title)
    titles.append(s_title)

    # scrapping all images
    images = image_scrap(s_driver)
    img_lst.append(images)
    

    # extractig meta data
    meta_data = meta_data_scrap(s_driver)
    meta_lst.append(meta_data)
        
    # click on show_more button
    show_more_click(s_driver)

    # extracting feature data
    feature = feature_scrap(s_driver)
    feature_lst.append(feature)
    
    # extracting description
    description = description_srcap(s_driver)
    desc_lst.append(description)
    
    cnt += 1
    print("=============================================")
    clear_output(wait=True)

Count:  1


In [ ]:
df_lst = [titles, img//g_lst, meta_lst,  feature_lst, desc_lst]
column_lst = ['Title', 'Images', 'Meta Data', 'Feature', 'Description']
df = pd.DataFrame (df_lst, columns = column_lst)
df.to_csv('Bikroy-Mobile-Category.csv', index=False)